In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
data = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        data.append(os.path.join(dirname, filename))
print(data)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[]


In [14]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

In [16]:
with open('Intent.json') as f:
    intents = json.load(f)

In [17]:
intents

{'intents': [{'intent': 'Greeting',
   'text': ['Hi',
    'Hi there',
    'Hola',
    'Hello',
    'Hello there',
    'Hya',
    'Hya there'],
   'responses': ['Hi human, please tell me your GeniSys user',
    'Hello human, please tell me your GeniSys user',
    'Hola human, please tell me your GeniSys user'],
   'extension': {'function': '', 'entities': False, 'responses': []},
   'context': {'in': '', 'out': 'GreetingUserRequest', 'clear': False},
   'entityType': 'NA',
   'entities': []},
  {'intent': 'GreetingResponse',
   'text': ['My user is Adam',
    'This is Adam',
    'I am Adam',
    'It is Adam',
    'My user is Bella',
    'This is Bella',
    'I am Bella',
    'It is Bella'],
   'responses': ['Great! Hi <HUMAN>! How can I help?',
    'Good! Hi <HUMAN>, how can I help you?',
    'Cool! Hello <HUMAN>, what can I do for you?',
    'OK! Hola <HUMAN>, how can I help you?',
    'OK! hi <HUMAN>, what can I do for you?'],
   'extension': {'function': 'extensions.gHumans.updateHum

Preprocessing Data

In [18]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [19]:
# get text and intent title from json data
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [20]:
inputs

['Hi',
 'Hi there',
 'Hola',
 'Hello',
 'Hello there',
 'Hya',
 'Hya there',
 'My user is Adam',
 'This is Adam',
 'I am Adam',
 'It is Adam',
 'My user is Bella',
 'This is Bella',
 'I am Bella',
 'It is Bella',
 'How are you?',
 'Hi how are you?',
 'Hello how are you?',
 'Hola how are you?',
 'How are you doing?',
 'Hope you are doing well?',
 'Hello hope you are doing well?',
 'Good thanks! My user is Adam',
 'Good thanks! This is Adam',
 'Good thanks! I am Adam',
 'Good thanks! It is Adam',
 'Great thanks! My user is Bella',
 'Great thanks! This is Bella',
 'Great thanks! I am Bella',
 'Great thanks! It is Bella',
 'What is my name?',
 'What do you call me?',
 'Who do you think I am?',
 'What do you think I am?',
 'Who are you talking to?',
 'What name do you call me by?',
 'Tell me my name',
 'What is your name?',
 'What could I call you?',
 'What can I call you?',
 'What do your friends call you?',
 'Who are you?',
 'Tell me your name?',
 'What is your real name?',
 'What is your

In [21]:
targets

['Greeting',
 'Greeting',
 'Greeting',
 'Greeting',
 'Greeting',
 'Greeting',
 'Greeting',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'NameQuery',
 'NameQuery',
 'NameQuery',
 'NameQuery',
 'NameQuery',
 'NameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery'

In [22]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [23]:
tokenizer.word_index

{'<unk>': 1,
 'you': 2,
 'are': 3,
 'is': 4,
 'i': 5,
 'what': 6,
 'me': 7,
 'the': 8,
 'do': 9,
 'can': 10,
 'your': 11,
 'am': 12,
 'tell': 13,
 'a': 14,
 'pod': 15,
 'bay': 16,
 'door': 17,
 'adam': 18,
 'bella': 19,
 'you?': 20,
 'thanks!': 21,
 'please': 22,
 'open': 23,
 'name?': 24,
 'real': 25,
 'not': 26,
 'prove': 27,
 'my': 28,
 'it': 29,
 'how': 30,
 'talking': 31,
 'thanks': 32,
 'to': 33,
 'gossip': 34,
 'why': 35,
 'good': 36,
 'call': 37,
 'name': 38,
 'know': 39,
 'see': 40,
 'hello': 41,
 'user': 42,
 'this': 43,
 'great': 44,
 'who': 45,
 'time?': 46,
 'thank': 47,
 "wasn't": 48,
 'bye': 49,
 'goodbye': 50,
 'very': 51,
 'self': 52,
 'aware': 53,
 'hi': 54,
 'there': 55,
 'me?': 56,
 'please?': 57,
 'time': 58,
 'ok': 59,
 'for': 60,
 'quiet': 61,
 'clever': 62,
 'girl': 63,
 'any': 64,
 'some': 65,
 'joke': 66,
 'will': 67,
 'have': 68,
 'conscious': 69,
 'hola': 70,
 'hya': 71,
 'hope': 72,
 'doing': 73,
 'well?': 74,
 'think': 75,
 'am?': 76,
 "what's": 77,
 'is?'

In [24]:
input_tensor

array([[ 0,  0,  0, ...,  0,  0, 54],
       [ 0,  0,  0, ...,  0, 54, 55],
       [ 0,  0,  0, ...,  0,  0, 70],
       ...,
       [ 0, 10,  2, ..., 52, 53, 22],
       [ 0, 10,  2, ..., 14, 69, 22],
       [ 0,  0,  0, ..., 68, 14, 69]])

In [25]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)



In [26]:
target_tensor

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [27]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (143, 9) and output shape: (143, 22)


In [28]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [29]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 512)         66048     
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               656384    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 22)                2838      
Total params: 758,166
Trainable params: 758,166
Non-trainable params: 0
_________________________________________________________________


In [30]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
5/5 [==============================] - 4s 38ms/step - loss: 3.1184 - accuracy: 0.0438
Epoch 2/50
5/5 [==============================] - 0s 47ms/step - loss: 2.7129 - accuracy: 0.2713
Epoch 3/50
5/5 [==============================] - 0s 45ms/step - loss: 1.8092 - accuracy: 0.4566
Epoch 4/50
5/5 [==============================] - 0s 45ms/step - loss: 1.2928 - accuracy: 0.6662
Epoch 5/50
5/5 [==============================] - 0s 44ms/step - loss: 0.6353 - accuracy: 0.7571
Epoch 6/50
5/5 [==============================] - 0s 35ms/step - loss: 0.4357 - accuracy: 0.8649
Epoch 7/50
5/5 [==============================] - 0s 36ms/step - loss: 0.2374 - accuracy: 0.8992
Epoch 8/50
5/5 [==============================] - 0s 37ms/step - loss: 0.0970 - accuracy: 0.9869
Epoch 9/50
5/5 [==============================] - 0s 40ms/step - loss: 0.1082 - accuracy: 0.9580
Epoch 10/50
5/5 [==============================] - 0s 41ms/step - loss: 0.0359 - accuracy: 0.9790
Epoch 11/50
5/5 [============

In [33]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

# chat with bot
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {} -- TYPE: {}'.format(res, typ))
    print()

Note: Enter 'quit' to break the loop.


You:  hi


Bot: Hello human, please tell me your GeniSys user -- TYPE: Greeting



You:  how are you


Bot: Hello, I am good thank you, how are you? Please tell me your GeniSys user -- TYPE: CourtesyGreeting



You:  quit
